Copyright © 2020, SAS Institute Inc., Cary, NC, USA.  All Rights Reserved.
SPDX-License-Identifier: Apache-2.0

# Create AWS Destinations with Group Credentials

In order to create an Amazon Web Services (AWS) destination with group credentials setup, the following steps must be completed:

1. Find a valid domain with AWS credentials.
2. Create credentials for a specified user and group.
3. Submit an API post to create an AWS destination. The delete destination API is also included.

Import packages for API calls, JSON handling, safe password handling, access key encryption, and printing readable JSON files. Set the SAS log in host.

In [ ]:
import requests
import json
import getpass
import base64
import pprint

host = "http://myserver.com"

### Administrator and AWS Login
Authenticate a token for logging into the host and for accessing an AWS domain.

In [ ]:
authURI = "/SASLogon/oauth/token"

headersAuth = {
    'accept': "application/json",
    'content-type': "application/x-www-form-urlencoded",
    'Authorization': "Basic c2FzLmVjOg=="}
authToken = ''
username = ''
password = ''
notAuthenticated = True

while notAuthenticated:
    username = input("Enter adminstrator user name: ")
    password = getpass.getpass(f"Enter password for user {username}: ")
    authBody = f"grant_type=password&username={username}&password={password}"
    authPost = requests.post(host + authURI,
                             data=authBody,
                             headers=headersAuth)
    if authPost.status_code == requests.codes.ok:
        authToken = authPost.json()['access_token']
        notAuthenticated = False
    else:
        print("Please enter a valid user name and password combination.")

password = ''

In order to successfully create an AWSkey domain, a username with access to retrieve the domain is required.

In [ ]:
domainUsername = input("Enter a user name with access to the domain information: ")
awsKeyID = input("Enter AWS key ID: ")
awsSecretKey = getpass.getpass("Enter AWS secret key: ")

### Domain Creation and Selection
Either create a new AWS domain or choose one from an existing list.

In [ ]:
headersGet = {
    'Authorization': 'Bearer ' + authToken}
headersDomainPut = {
    'Content-Type': "application/vnd.sas.credential.domain+json",
    'Accept': "application/vnd.sas.credential.domain+json",
    'Authorization': 'Bearer ' + authToken}
headersCredentialPut = {
    'Content-Type': "application/vnd.sas.credential+json",
    'Accept': "application/vnd.sas.credential+json",
    'Authorization': 'Bearer ' + authToken}

#### Create a New Domain

In [ ]:
domainName = input("Enter new domain name: ")
domainURL = host + "/credentials/domains/"
domainAttributes = {
    'id': domainName,
    'type': 'password',
    'description': "AWS credentials"}

domainPut = requests.put(domainURL + domainName,
                         data=json.dumps(domainAttributes),
                         headers = headersDomainPut)
print(domainPut)

#### Get a List of Domains and Select One

In [ ]:
domainURL = host + "/credentials/domains/"
domain1000URLs = domainURL + "?limit=1000"
domainGet = requests.get(domain1000URLs, headers=headersGet)

for i, domain in enumerate(domainGet.json()['items']):
    print(f"{i}. Domain ID: {domain['id']}, {domain['type']}")

validDomain = False
while not validDomain:
    domainIndex = input("Select a domain number from the list: ")
    try:
        domainID = domainGet.json()['items'][int(domainIndex)]['id']
        print(domainID)
        domainName = domainID
        validDomain = True
    except:
        print("Please select a valid domain number from the list.")

### Create User and Group Credentials
User credentials use the identityType of 'user', which is the same as the SAS Open Model Manager user name.

In order to create an AWSKey domain, a group ID is required to retrieve the domain information.

In [ ]:
credentialURL = domainURL + domainName + '/users/' + domainUsername
print(credentialURL)

encodedAccessKey = str(base64.b64encode(awsSecretKey.encode('utf-8')), 'utf-8')

credentialAttributes = {
    'domainId': domainName,
    'identityType': 'user',
    'identityId': domainUsername,
    'domainType': 'password',
    'properties': {'userId': awsKeyID},
    'secrets': {'password': encodedAccessKey}}

credentialPut = requests.put(credentialURL,
                             data=json.dumps(credentialAttributes),
                             headers=headersCredentialPut)
print(credentialPut)

In [ ]:
domainGroupID = input("Enter a group ID with access to domain informatin: ")

credentialURL = domainURL + domainName + '/groups/' + domainGroupID
print(credentialURL)

encodedAccessKey = str(base64.b64encode(awsSecretKey.encode('utf-8')), 'utf-8')

credentialAttributes = {
    'domainId': domainName,
    'identityType': 'group',
    'identityId': domainGroupID,
    'domainType': 'password',
    'properties': {'userId': awsKeyID},
    'secrets': {'password': encodedAccessKey}}

credentialPut = requests.put(credentialURL,
                             data=json.dumps(credentialAttributes),
                             headers=headersCredentialPut)
print(credentialPut)

To verify the group credentials, check at `/credentials/domains/<domainName>/secrets?lookupInGroup=true`.

### Create or Delete an AWS Destination
Generate a list of domain users, then create an AWS destination, verify its creation, and delete it.

In [ ]:
credentialURL = domainURL + domainName + '/credentials'
print(credentialURL)

credentialGet = requests.get(credentialURL, headers=headersGet)
print(credentialGet)

for i, domain in enumerate(credentialGet.json()['items']):
    print(f"{i}. Domain ID: {domain['domainId']}, {domain['identityId']}, {domain['identityType']}")

#### Create a Destination

In [ ]:
destinationName = input("Enter destination name: ")
destinationDesc = input("Enter description for destination {destinationName}: ")
awsRegion = input("Enter AWS region: ")
k8sClusterName = input("Enter AWS kubernetes cluster name: ")

targetDestination = {
    'name': destinationName,
    'destinationType': 'aws',
    'description': destinationDesc,
    'properties': [{'name': 'domainId', 'value': domainName},
                   {'name': 'region', 'value': awsRegion},
                   {'name': 'kubernetesCluster', 'value': k8sClusterName}]}

headersPost = {
    'Content-Type': "application/vnd.sas.models.publishing.destination.aws+json",
    'Authorization': 'Bearer ' + authToken}

destinationPost = requests.post(host + '/modelPublish/destinations',
                                data=json.dumps(targetDestination),
                                headers=headersPost)
print(destinationPost)

#### Verify that the Destination was Created

In [ ]:
destinationGet = requests.get(host + "/modelPublish/destinations/" + destinationName,
                              headers=headersGet)
print(destinationGet)
pprint.pprint(destinationGet.json())

#### Delete the Created Destination

In [ ]:
headersDelete={'Authorization': 'Bearer ' + authToken}
destinationDelete = requests.delete(host + "/modelPublish/destinations/" + destinationName,
                                    headers = headersDelete)
print(destinationDelete)